# Construction des dictionnaires pour la Correction Lexicale

Dans ce notebook sera présenté le pré-traitement du jeu de donnée [Lexique3](https://chrplr.github.io/openlexicon/datasets-info/) afin d'obtenir des données utilisable par notre algorithme de correction lexicale.

Ce pré-traitement de donnée aura plusieurs objectifs :
- Nettoyer le jeu de données de toutes ses impuretés (doublons, accents, etc...)
- Augmenter la vitesse de notre algo de correction lexicale en divisant le dataset

**Pour exécuter le notebook, mettre le dossier xlsx de Lexique3 dans le dossier dataset en le nommant liste_de_mots.xlsx**

## Importation des ressources nécessaires

### Importation des dépandances.

In [1]:
from pathlib import Path
from unidecode import unidecode
import pandas as pd
import matplotlib.pyplot as plt

### Importation du dataset en format excel


In [ ]:
data = pd.read_excel(io="../dataset/liste_de_mots.xlsx", dtype=str)
print(data.head(10))

## Pré-traitement des données pour la création du dataset

### Organisation du dataset

Récupération des colonnes utiles, dans notre cas nous avons juste besoin de la colonne **ortho** pour **orthographe** des mots.

In [ ]:
dataset =  pd.DataFrame(data["ortho"], dtype=str)
dataset.drop_duplicates(keep = 'first', inplace=True) # Supression des doublons
print(dataset.head(10))

### Nettoyage du dataset

#### Suppression des accents et autres caractères français compliquant l'utilisation du jeu de données

In [ ]:
print("Avant conversion")
print(dataset.tail())

Opération de conversion des str avec accents en str sans accents

In [ ]:
dataset = dataset.applymap(unidecode)
dataset.head(10)

print("Après conversion")
print(dataset.tail())

### Triage le dataset

Le tri du dataset par nombre de lettre aura pour but de séparer le dataset en sous fichier pour ainsi augmenter la vitesse de calcule de "la distance de Levensthein" en sélectionnant ces fichers en fonctions du mot demandé

#### Analyse de la répartition des mots par nombre de caractères dans le dataset

Pour cela, on crée une nouvelle colonne nb_caractères qui contiendra le nombre de caractères contenue dans chaque mots en fonction de la ligne de celui-ci

In [ ]:
dataset_nb_caracteres = dataset.applymap(len)
dataset_nb_caracteres.head()

Création du nouveau dataframe "dataset_3" contenant comme colonne:
-mot
-nombre de caractères

In [ ]:
dataset_2 = pd.DataFrame({"mot": dataset["ortho"], "nb_caracteres": dataset_nb_caracteres["ortho"]})
dataset_2.head()

Affichage de l'histogramme de la répartition du nombre de mots en fonction du nombre de caractère.

In [ ]:
dataset_2["nb_caracteres"].value_counts().plot(kind="bar")

#### Regroupement des mots par nombre de caractère.

On regroupe les mots par nombre de caractère afin d'obtenir une répartition plus uniformes des ensembles de mots

In [ ]:
dict_dataset_mots_par_nb_carac = {}
for i in range(1, 26):
    df = dataset_2 # Varaible intermédiaire pour simplifier l'écriture de la ligne suivante
    dict_dataset_mots_par_nb_carac[i] = df[df["nb_caracteres"] == i]["mot"] # On sélectionne uniquement la colonne des mots; plus besoin de la colonne du nombre de caractères

dict_dataset_mots_par_nb_carac[1]

#### Regroupement des catégories où il y a peu de mots ensemble

Dans notre cas on va créer une nouvelle catégorie qui va reprendre tous les mots ayant les nombres de caractères ci-dessous:
**(14, 4, 15, 3, 16, 17, 2, 18, 19, 1, 20, 21, 22, 23, 24, 25)**.

In [ ]:
groupe_autre = [14, 4, 15, 3, 16, 17, 2, 18, 19, 1, 20, 21, 22, 23, 24, 25]
dataset_autre = pd.DataFrame(dtype=str)

for i in groupe_autre:
    dataset_autre = pd.concat((dataset_autre, dict_dataset_mots_par_nb_carac[i]))
dataset_autre.columns = ["mot"]
dataset_autre

##### Création du dataset final dans un dictionnaire

In [ ]:
dict_final_dataset = {}
dict_final_dataset["autre"] = dataset_autre

for i in range(1, 26):
    if i not in groupe_autre:
        dict_final_dataset[i] = dict_dataset_mots_par_nb_carac[i]

##### Affichage de la répartition des ensembles de  mots

In [ ]:
dict_final_dataset_repartion_mots = {}

for key, value in dict_final_dataset.items():
    dict_final_dataset_repartion_mots[key] = len(value.index)

print(dict_final_dataset_repartion_mots)
plt.bar(range(len(dict_final_dataset_repartion_mots)), list(dict_final_dataset_repartion_mots.values()), align='center')
plt.xticks(range(len(dict_final_dataset_repartion_mots)), list(dict_final_dataset_repartion_mots.keys()))

##### Sauvegarde du dataset sur le PC

In [ ]:
SAVE_PATH = Path("../dataset/")
for key in dict_final_dataset.keys():
    dict_final_dataset[key].to_csv(SAVE_PATH / f"mots_{key}.csv", index=False)